In [1]:
%matplotlib qt

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.animation import FuncAnimation

from scipy.stats import skewnorm, norm
import arviz as az

In [3]:
from matplotlib.legend_handler import HandlerPatch


class HandlerCircle(HandlerPatch):
    def create_artists(self, legend, orig_handle,
                       xdescent, ydescent, width, height, fontsize, trans):
        center = 0.5 * width - 0.5 * xdescent, 0.5 * height - 0.5 * ydescent
        radius = min(width + xdescent, height + ydescent)
        p = Circle(xy=center, radius=radius)
        self.update_prop(p, orig_handle, legend)
        p.set_transform(trans)
        return [p]

In [4]:
rng = np.random.default_rng(10)
x = skewnorm(3, loc=5, scale=2).rvs(size=50, random_state=rng)
y = - np.ones_like(x)

## Computing dotplot animation

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from functools import partial

ndots = 20
qlist = np.linspace(1 / (2 * ndots), 1 - 1 / (2 * ndots), ndots)
values = np.quantile(x, qlist)

binwidth = np.sqrt((values[-1] - values[0] + 1) ** 2 / (2 * ndots * np.pi))
radius = binwidth / 2

stack_locs, stack_count = az.plots.dotplot.wilkinson_algorithm(values, binwidth)
c_x, c_y = az.plots.dotplot.layout_stacks(stack_locs, stack_count, binwidth, 1, False)

fig, ax = plt.subplots()
rugs = [ax.plot([], [], "|k", markersize=10)[0] for _ in x]
quantiles = [ax.plot([], [], "|b", markersize=10)[0] for _ in values]
dots = [Circle(xy, radius, color="blue") for xy in zip(c_x, c_y)]
window, = ax.plot([], [], "r-", linewidth=2)
for c in dots:
    ax.add_patch(c)

ax.legend(
    handles=[dots[0], window, quantiles[0], rugs[0]],
    labels=["Gràfic de punts", "Amplada de classe", "Quantils", "Dades"],
    handler_map={Circle: HandlerCircle()},
)

def init():
    ax.set_title('Gràfic de punts per quantils')
    ax.spines.left.set_color('none')
    ax.spines.right.set_color('none')
    ax.spines.bottom.set_position('zero')
    ax.spines.top.set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.tick_params(left=False, labelleft=False)
    ax.set_ylim(-0.45, 3)
    ax.set_xlim(x.min()-0.1, x.max()+0.1)
    ax.set_aspect("equal")
    for r in rugs:
        r.set_data([], [])
    for q in quantiles:
        q.set_data([], [])
    for c in dots:
        c.set(color="none")
    window.set_data([], [])
    return tuple([window, *rugs, *quantiles, *dots])

def update(frame, samples, qs):
    step, idx = frame
    if step == "rug":
        for rug, sample in zip(rugs, samples):
            rug.set_data([sample], [-0.4])
            rug.set_color("black")
    elif step == "quantiles":
        for q, val in zip(quantiles, values):
            q.set_data([val], [-0.2])
            q.set_color("blue")
    elif step == "dot":
        center_x = c_x[idx]
        indexes, = np.nonzero((qs >= (center_x - radius)) & (qs < (center_x + radius)))
        for i in indexes:
            quantiles[i].set(color="red", markersize=30)
        window.set_data([qs[indexes[0]], qs[indexes[0]] + binwidth], [0, 0])
        dots[idx].set_color("blue")
        if idx > 0:
            oldcenter = c_x[idx-1]
            if not np.isclose(oldcenter, center_x):
                indexes, = np.nonzero((qs >= (oldcenter - radius)) & (qs < (oldcenter + radius)))
                for i in indexes:
                    quantiles[i].set(color="lightgray", markersize=10)
    elif step == "cleanup":
        center_x = c_x[-1]
        indexes, = np.nonzero((qs >= (center_x - radius)) & (qs < (center_x + radius)))
        for i in indexes:
            quantiles[i].set(color="lightgray", markersize=10)
        window.set_data([], [])
    return tuple([window, *rugs, *quantiles, *dots])

frames = [
    *[("wait", None)]*2,
    ("rug", None),
    *[("wait", None)]*2,
    ("quantiles", None),
    *[("wait", None)]*2,
    *[("dot", i) for i in range(ndots)],
    ("cleanup", None),
    *[("wait", None)]*6,
]

sorted_x = np.sort(x)
ani = FuncAnimation(
    fig, partial(update, samples=x, qs=values),
    frames=frames,
    init_func=init,
    blit=True,
    interval=300,
)

#ani.save("dotplot.ca.mp4", dpi=300)

with open("dotplot.ca.html", mode="w", encoding="utf-8") as f:
    f.write(ani.to_jshtml())

plt.show()

##